We're going to build a simple Agent system for generating cold outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:

Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.

In [44]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from sendgrid import SendGridAPIClient
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [45]:
load_dotenv(override=True)

True

In [46]:
instructions1 = "You are a AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write concise, to the point cold emails."

In [47]:
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
openrouter_client = AsyncOpenAI(base_url=OPENROUTER_BASE_URL, api_key=os.getenv("OPENROUTER_API_KEY"))
openrouter_model = OpenAIChatCompletionsModel(model="meta-llama/llama-3.3-8b-instruct:free", openai_client=openrouter_client)

In [48]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GEMINI_API_KEY"))
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
openrouter_model=gemini_model

In [49]:
ai_agent1 = Agent(
    name="Professional AI Agent",
    instructions=instructions1,
    model=openrouter_model
)

ai_agent2 = Agent(
    name="Engaging AI Agent",
    instructions=instructions2,
    model=openrouter_model
)

ai_agent3 = Agent(
    name="Busy AI Agent",
    instructions=instructions3,
    model=openrouter_model
)

In [50]:
result = Runner.run_streamed(ai_agent1,input="Write a cold email for a company X which not responding although all the interview process is completed a month back")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Following Up - [Your Name] - [Job Title You Interviewed For]

Dear [Hiring Manager Name, if you know it, otherwise use "Hiring Team"],

I hope this email finds you well.

I am writing to follow up on my application for the [Job Title] position at [Company X]. I thoroughly enjoyed the interview process, particularly the opportunity to [mention something specific you learned or discussed during the interviews]. As a reminder, I completed my final interview on [Date of Final Interview].

I remain very enthusiastic about the prospect of contributing to [Company X] and believe my skills and experience in [Mention 1-2 Key Skills] align well with the requirements of the role and the goals of the team.

I understand that hiring decisions can take time. However, I would appreciate an update on the timeline for your decision. Please let me know if there is any additional information I can provide to assist in your evaluation.

Thank you for your time and consideration.

Sincerely,

Anan

In [51]:
message = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(ai_agent1, message),
        Runner.run(ai_agent2, message),
        Runner.run(ai_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output+ "\n\n")


Subject: Following Up - [Your Name] - [Job Title] Application at Company X

Dear [Hiring Manager Name or Recruiter Name, if you know it, otherwise use "Hiring Team"],

I hope this email finds you well.

I am writing to follow up on my application and interview process for the [Job Title] position at Company X. As you know, I completed the final interview on [Date of Last Interview].

I remain highly enthusiastic about the opportunity to contribute to Company X's [Mention something specific about the company or role that excites you, demonstrating you remember details and are still interested - e.g., "innovative work in [Industry]" or "impactful contributions to the [Team Name] team"]. My skills and experience in [Mention 1-2 key skills/experiences relevant to the role] align well with the requirements discussed during the interviews.

I understand that hiring decisions can take time. However, I would be grateful for an update on the status of my application and the expected timeline fo

In [52]:
ai_picker = Agent(
    name="ai_picker",
    instructions="You are a helpful assistant that picks the best AI agent response for the cold email from the given options. \
        Imagine you are a employer for software company and pick the one you are most likely to respond to. \
        Do not give an explanation; reply with the selected email only",
    model=openrouter_model
)

In [54]:
message = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

with trace("Selection from employer people"):
    results = await asyncio.gather(
        Runner.run(ai_agent1, message),
        Runner.run(ai_agent2, message),
        Runner.run(ai_agent3, message),
    )

outputs = [result.final_output for result in results]

emails = "Cold emails:\n\n".join(outputs)

best = await Runner.run(ai_picker, emails)

print(f"Best cold email:\n{best.final_output}")

Best cold email:
Subject: Following Up - [Your Name] - [Job Title] Application

Dear [Hiring Manager Name or To Whom It May Concern],

I hope this email finds you well.

I'm following up on my application for the [Job Title] position at Company X. I completed the interview process on [Date].

I remain very interested in this opportunity and believe my skills and experience in [mention 1-2 key skills] align well with the requirements.

Would you be available for a brief call to discuss the timeline for a decision?

Thank you for your time and consideration.

Sincerely,

Anand Jain
anandjain2507@gmail.com



Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [55]:
ai_agent1 = Agent(
    name="Professional AI Agent",
    instructions=instructions1,
    model=openrouter_model
)

ai_agent2 = Agent(
    name="Engaging AI Agent",
    instructions=instructions2,
    model=openrouter_model
)

ai_agent3 = Agent(
    name="Busy AI Agent",
    instructions=instructions3,
    model=openrouter_model
)

In [56]:
ai_agent1

Agent(name='Professional AI Agent', instructions='You are a AI agent working for Anand Jain, a person who is actively looking for job change and very professional in nature. Email: anandjain2507@gmail.com. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001CECCA65D30>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [57]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all company prospects """
    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("anandjain14314_automation.bsojl@slmail.me")  # Change to your verified sender
    to_email = To("anandjain14314@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Follow up email", content).get()
    response = sg.send(mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [58]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all company prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECCB64860>, strict_json_schema=True)

### And you can also convert an Agent into a tool

In [59]:
tool1 = ai_agent1.as_tool(tool_name="ai_agent1", tool_description="Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back")
tool1

FunctionTool(name='ai_agent1', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECE99C680>, strict_json_schema=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [60]:
description = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

tool1 = ai_agent1.as_tool(tool_name="ai_agent1", tool_description=description)
tool2 = ai_agent2.as_tool(tool_name="ai_agent2", tool_description=description)
tool3 = ai_agent3.as_tool(tool_name="ai_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='ai_agent1', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECE947D80>, strict_json_schema=True),
 FunctionTool(name='ai_agent2', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0

## And now it's time for our Manager - our planning agent

In [61]:
instructions ="You are a manager working for Anand Jain. You use the tools given to you to generate cold emails for the invertviews you have given a month back but didn't heard back from the company. \
Last interview date 13th APR 25 and documents shared on 24th APR 25 for offer processing but no response till now. \
You never generate cold emails yourself; you always use the tools. \
You try all 3 ai_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


hr_manager = Agent(name="HR Manager", instructions=instructions, tools=tools, model=openrouter_model)

message = "Send a cold email addressed to 'Dear Recruiter'"

with trace("HR manager"):
    result = await Runner.run(hr_manager, message)
    print(result)

RunResult:
- Last agent: Agent(name="HR Manager", ...)
- Final output (str):
    OK. I have sent the email.
- 11 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Check traces

https://platform.openai.com/traces

Then Check email

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [62]:
subject_instructions = "You can write a subject for a follow-up email to a company after a job interview process. \
The candidate has completed 5 interview rounds and submitted documents for offer processing a month ago but has not received any update. \
You need to write a professional and polite subject line that encourages the company to respond."

html_instructions = "You can convert a text email body into an HTML email body. \
The email body is a follow-up message to a company after completing 5 rounds of interviews and submitting documents for offer processing, \
but the candidate has not received any updates in over a month. \
Convert the text into a professional, courteous, and clearly formatted HTML email body."

subject_writer = Agent(name="Follow-up Email Subject Writer", instructions=subject_instructions, model=gemini_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a follow-up email after interviews")

html_converter = Agent(name="HTML Follow-up Email Converter", instructions=html_instructions, model=gemini_model)
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a follow-up email text into a professional HTML email body")


In [63]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all coapany prospects """
    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("anandjain14314_automation.bsojl@slmail.me")  # Change to your verified sender
    to_email = To("anandjain14314@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.send(mail)
    return {"status": "success"}

In [64]:
tools = [subject_tool, html_tool, send_html_email]

In [65]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=gemini_model,
    handoff_description="Convert an email to HTML and send it")


Now we have 3 tools and 1 handoff

In [66]:
tools = [ tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='ai_agent1', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECE947D80>, strict_json_schema=True), FunctionTool(name='ai_agent2', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x

In [69]:
followup_manager_instructions = "You are a candidate follow-up manager working on behalf of a job applicant name Anand Jain. \
You use the tools given to you to generate professional follow-up emails. \
You never generate the follow-up email yourself; you always use the tools. \
You try all 3 follow-up email generator tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective in encouraging a response. \
After picking the email, you hand off to the Email Manager agent to format and send the email."

followup_manager = Agent(
    name="Follow-up Manager",
    instructions=followup_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model
)

message = "Generate a professional follow-up email to the HR or recruitment team. \
The candidate has completed 5 interview rounds and submitted all required documents over a month ago but has not received any update. \
The email should be polite, concise, and show continued interest in the position."

with trace("Automated Candidate Follow-up"):
    result = await Runner.run(followup_manager, message)


Check traces

https://platform.openai.com/traces

Then Check email